# Modelo Hull-White

Veamos la construcción de la función Theta, la simulación y el cálculo de los factores de descuento con las fórmulas del modelo.

In [1]:
# Sirve para importar los datos de una curva
import various_functions as vf

In [2]:
# Obtenemos los valores de la curva
curva = vf.get_curva('./curva_3.csv')
for x in zip(curva[0], curva[1]):
    print(x)

(0.002739726, 0.999975001)
(0.019178082, 0.999823573)
(0.038356164, 0.999647169)
(0.060273973, 0.999450913)
(0.087671233, 0.999201527)
(0.167123288, 0.998473099)
(0.252054795, 0.997633801)
(0.336986301, 0.996740906)
(0.419178082, 0.99584806)
(0.509589041, 0.994859891)
(0.75890411, 0.991800113)
(1.0, 0.988563827)
(1.506849315, 0.981183423)
(2.005479452, 0.972693527)
(3.002739726, 0.954199287)
(4.002739726, 0.934493447)
(5.002739726, 0.914165264)
(7.010958904, 0.851735228)
(10.00547945, 0.781818476)
(12.00821918, 0.736803541)
(15.0109589, 0.674157693)
(20.01369863, 0.582462093)
(25.01917808, 0.506693422)
(30.02465753, 0.442322476)
(40.02739726, 0.341431864)
(50.03287671, 0.267091985)


In [3]:
# La curva está expresada en df. La pasamos a tasa.
import math
from bokeh.plotting import figure, show, output_notebook

tenors = []
rates = []
for x in zip(curva[0], curva[1]):
    # df = exp(-rate * yf)
    # log(df) = - rate * yf
    # - log(df) / yf = rate
    
    # yf = x[0] / 365
    rate = - math.log(x[1]) / x[0]
    tenors.append(x[0])
    rates.append(rate)

# Graficamos la curva
def plot_curve(tenors, rates):
    # se define la data
    x = tenors
    y = rates

    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title="Curva Cupón Cero", x_axis_label='Tiempo', y_axis_label='Tasa')

    # add a line renderer with legend and line thickness
    p.line(x, y, legend="Curva", line_width=2)

    # show the results
    show(p)
for x in zip(tenors, rates):
    print(x)
plot_curve(tenors, rates)


(0.002739726, 0.00912474914651258)
(0.019178082, 0.009200219556570011)
(0.038356164, 0.009200431500461993)
(0.060273973, 0.009112354406638039)
(0.087671233, 0.009111220659561416)
(0.167123288, 0.009143357096418869)
(0.252054795, 0.009398761379409495)
(0.336986301, 0.009687089369730808)
(0.419178082, 0.00992557438942472)
(0.509589041, 0.010112785773117403)
(0.75890411, 0.010849448414305027)
(1.0, 0.011502068906166132)
(1.506849315, 0.012606344357234234)
(2.005479452, 0.013805289190988801)
(3.002739726, 0.015613318969096032)
(4.002739726, 0.016926072903784863)
(5.002739726, 0.01793895241083197)
(7.010958904, 0.022889816914631645)
(10.00547945, 0.02459978999406266)
(12.00821918, 0.0254354108262627)
(15.0109589, 0.026266891554823073)
(20.01369863, 0.027006061300658836)
(25.01917808, 0.0271731208136153)
(30.02465753, 0.02716820594525686)
(40.02739726, 0.02684679027875789)
(50.03287671, 0.02638589370857481)


Loading BokehJS ...

Recordemos la expresión para Theta:

La fórmula para la función $\theta_{t}$ es:

$$\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\big(1-\exp(-2\gamma^*t)\big)$$

Necesitamos una representación en tiempo contínuo de la curva para poder calcular $f(0,t)$ y su derivada. Para eso, vamos a utilizar un **cubic-spline.**

In [4]:
# Definimos la curva como un spline cúbico
from scipy.interpolate import interp1d
zrate = interp1d(tenors, rates, 'cubic', fill_value="extrapolate")

In [5]:
# Veamos un gráfico
rates_spline = []
tenors_spline = []
steps_per_year = 264.0
dt = 1 / steps_per_year
years = 20
for i in range(0, int(steps_per_year * years)):
    tenors_spline.append(i * dt)
    rates_spline.append(zrate(i * dt))
plot_curve(tenors_spline, rates_spline)

Loading BokehJS ...

In [6]:
# Definimos la derivada primera y segunda como diferencias finitas
def dzrate(t):
    delta = .00000001
    return (zrate(t + delta) - zrate(t - delta)) / (2 * delta)

def d2zrate(t):
    delta = .0001
    return (zrate(t + delta) - 2 * zrate(t) + zrate(t - delta)) / (delta**2)

t = curva[0][1]
print(zrate(t))
print (dzrate(t))
print(d2zrate(t))

0.009200219556570013
0.0026649545417645193
-0.25392563347131913


Sabemos que:

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$,

y también que:

$$\frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$$

In [7]:
# Podemos ahora definir las funciones fwd(0,t) y dfwd(0,t)
def fwd(t):
    return zrate(t) + t * dzrate(t)

def dfwd(t):
    return 2 * dzrate(t) + t * d2zrate(t)

print(fwd(t))
print(dfwd(t))

0.009251328273298245
0.0004601024629141354


In [8]:
# Con esto, podemos ya escribir una expresión para la función Theta.
# Definamos un valor para sigma y gamma
sigma = .015
gamma = .5
r0 = rates_spline[0]

def theta(t):
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux
print(theta(t))

0.005090040553894464


$$\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\big(1-\exp(-2\gamma^*t)\big)$$

In [9]:
tiempos = []
thetas = []
dt = 1 / 264.0
for i in range(0, 66):
    t = i * dt
    tiempos.append(t)
    thetas.append(theta(t))
    
plot_curve(tiempos, thetas)

Loading BokehJS ...

In [10]:
# Finalmente, podemos simular.
from numpy import random as rnd
def sim_hw(gamma, sigma, theta, r0, num_steps, seed = None):
    r = r0
    dt = 1 / 264.0
    sqdt = math.sqrt(dt)
    sim = [r0]
    rnd.seed(seed)
    for i in range(1, num_steps):
        # print(str(r))
        epsilon = rnd.normal()
        r = r + (theta((i - 1) * dt) - gamma * r) * dt + sigma * sqdt * epsilon
        sim.append(r)
        # print(str(theta((i - 1) * dt)) + "\t" + str(epsilon) + "\t" + str(r) )
    return sim

In [11]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    dt = 1/264.0
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)
    # for i in range(0, 5000):
    #    rnd.normal()
    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    for i in range(0, num_steps):
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            # print(str(r))
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
            # print(str(theta_array[j - 1]) + "\t" + str(alea[i][j - 1]) + "\t" + str(r) )
    return sim

In [12]:
def plot_simulation(dt, sim):
    # se define el eje tiempo
    tiempo = [0]
    for i in range(1, len(sim)):
        tiempo.append(i * dt)

    # output hacia el notebook
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title="Simulación", x_axis_label='Tiempo', y_axis_label='Tasa')

    # add a line renderer with legend and line thickness
    p.line(tiempo, sim, legend="Curva", line_width=2)

    # show the results
    show(p)

In [15]:
dt = 1 / 264.0
sim = sim_hw(gamma, sigma, theta, r0, 264, 2000)
plot_simulation(dt, sim)

Loading BokehJS ...

In [16]:
print(sim[263])

0.025035064292863295


In [17]:
# Coeficiente B del modelo de HW
def b_hw(gamma, t, T):
    """
    gamma : para 
    sigma: 
    t : plazo de....
    T: plazo ....
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

In [18]:
# Coeficiente A del modelo de HW
def a_hw(zrate, fwd, gamma, sigma, t, T, verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [19]:
# Factor de descuento según HW
print("sigma: " + str(sigma))
print("gamma: " + str(gamma))
def zero_hw(r, gamma, sigma, zrate, fwd, t, T):
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

sigma: 0.015
gamma: 0.5


In [20]:
# Verifiquemos los valores que entrega la fórmula de HW con los datos de la curva
r0 = zrate(1/365000)
print("r0: " + str(r0))
t = 0
T = curva[0][15]
z = zero_hw(r0, gamma, sigma, zrate, fwd, t, T)
print("zero_hw: " + str(round(z, 8)))
print("zero curva: " + str(round(curva[1][15], 8)))

r0: 0.009106632904622951
zero_hw: 0.93449342
zero curva: 0.93449345


Vamos a valorizar, en 1Y más, un bono semestral a tasa fija del 5% que en 1Y más le quedarán 4Y. Podemos además hacer el ejercicio de varificar la simulación calculando los valores de los bonos cero cupón usando la simulación.

In [21]:
first_coupon = 1.5
tasa = .05
num_cupones = 8
bf = vf.bono_tasa_fija(first_coupon, num_cupones, tasa)
bf

([1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
 [0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 1.025])

In [23]:
r1Y = sim[263]
r1Y

0.025035064292863295

In [24]:
# ¿Cuál es la curva en t = 1Y en esta simulación?
t = 1
zeros_1Y = []
for plazo in bf[0]:
    zeros_1Y.append(zero_hw(r1Y, gamma, sigma, zrate, fwd, t, plazo))
print(zeros_1Y)

[0.9877469302777236, 0.9754094990622397, 0.9633538376284455, 0.951677331187178, 0.9401434233564852, 0.9289688672915929, 0.9183286175740412, 0.9069594665912871]


In [25]:
# ¿Cuál es el valor presente?
import numpy as np
vp = np.dot(zeros_1Y, bf[1])
print("vp: " + str(vp))

vp: 1.0962741659155117


In [26]:
# Chequear que dot haga lo que creemos que está haciendo
vp1 = 0
for y in zip(bf[1], zeros_1Y):
    vp1 += y[0] * y[1]
vp1

1.0962741659155117

In [28]:
num_sim = 10
def sim_many(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf):
    result = []
    for i in range(0, num_sim):
        r1y = sim_hw(gamma, sigma, theta, r0, 264)[263]
        zeros_1Y = []
        for plazo in bf[0]:
            zeros_1Y.append(zero_hw(r1y, gamma, sigma, zrate, fwd, t, plazo))
        vp = np.dot(zeros_1Y, bf[1])
        result.append(vp)
    return result

res = sim_many(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf)
res

[1.1279540277552287,
 1.1017071334219528,
 1.109978338654758,
 1.129794026792734,
 1.1089066369872622,
 1.1037275422507045,
 1.090911338993188,
 1.1169754223503474,
 1.1028002001274457,
 1.1181127285470778]

In [29]:
num_sim = 1000
# dfs = 0
t = .25
sim = 0
seed = None
num_steps = int(t*264)
print("num_steps: " + str(num_steps))
for i in range(0, num_sim):
    sim += math.exp(np.sum(- dt * np.array(
        sim_hw(gamma, sigma, theta, r0, num_steps, seed))))
    # dfs += math.exp(np.sum(sim))
ez = sim / num_sim
z_curva = math.exp(-zrate(t) * t)
print("ez: " + str(ez))
print("z_curva: " + str(z_curva))

num_steps: 66
ez: 0.997597204287458
z_curva: 0.9976549482230134


In [31]:
num_sim = 50000
t = .25
num_steps = int(t * 264)
print("num_steps: " + str(num_steps))
seed = None
df = 0
s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
    # print(len(sim))
    df += math.exp(-dt * np.sum(sim))
ez = df / num_sim
z_curva = math.exp(-zrate(t) * t)
print("ez: " + str(ez))
print("z_curva: " + str(z_curva))
print(t)

num_steps: 66
ez: 0.9976520059005399
z_curva: 0.9976549482230134
0.25
